# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [20]:
from data_loader import MyDataLoader
%load_ext autoreload 
%autoreload 2
dl = MyDataLoader("combine")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
dl.normalize_train_data(grayScale=True)

In [10]:
dl.train_test_split(0.2)

In [21]:
X_train, y_train, X_test, y_test = dl.to_numpy_array()

TypeError: Cannot interpret '<built-in function array>' as a data type

In [14]:
from PIL import Image
import numpy as np

img = Image.open("test.jpg").convert('L')
img = img.resize((500,500))
img.show()
arr = np.array(img)

In [13]:
arr.shape

(500, 500)